# Desarrollo del proyecto segun los pasos 

In [ ]:
import pandas as pd 
from sqlalchemy import create_engine, text
import json

url1 = 'https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv'

url2 = 'https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv'


movie_df = pd.read_csv(url1)
credits_df = pd.read_csv(url2)

In [ ]:
movie_df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')


In [ ]:
credits_df.columns

Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')


In [ ]:
#configuro la conexion de la Bdd
db_url = 'postgresql://gitpod@localhost:5432/movie_db'
eng = create_engine(db_url)

#Esto lo guarda (recuerda)
movie_df.to_sql('movies', eng, if_exists='replace', index=False)
credits_df.to_sql('credits', eng, if_exists='replace', index=False)


In [ ]:
with eng.connect() as connection:

    connection.execute(text("DROP TABLE IF EXISTS unifica_movies;"))
    connection.commit()
    

In [ ]:
#Conexion a la Bdd 
with eng.connect() as connection:
    
    query = text("""CREATE TABLE unifica_movies AS
        SELECT 
            m.id AS movie_id,
            m.title,
            m.overview,
            m.genres,
            m.keywords,
            c.cast,
            c.crew
        FROM movies m 
        JOIN credits c 
        ON m.title = c.title;
        """)
    connection.execute(query)
    connection.commit()



In [ ]:
#aqui ya unificamos las bdd
dfUnificada = pd.read_sql_table('unifica_movies', eng)

print(dfUnificada.columns)

Index(['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew'], dtype='object')


In [ ]:
#conversion Json a Python

dfUnificada['genres'] = dfUnificada['genres'].apply(json.loads)
dfUnificada['keywords'] = dfUnificada['keywords'].apply(json.loads)
dfUnificada['cast'] = dfUnificada['cast'].apply(json.loads)
dfUnificada['crew'] = dfUnificada['crew'].apply(json.loads)

In [ ]:
def extrae_names(json_list):
    return [item['name'] for item in json_list]

dfUnificada['genres'] = dfUnificada['genres'].apply(extrae_names)
dfUnificada['keywords'] = dfUnificada['keywords'].apply(extrae_names)

Traceback (most recent call last):
  File "/home/vscode/.vscode-remote/extensions/ms-python.python-2025.0.0-linux-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 4, in <module>
  File "/workspaces/KNN-project/.venv/lib/python3.10/site-packages/pandas/core/series.py", line 4924, in apply
    ).apply()
  File "/workspaces/KNN-project/.venv/lib/python3.10/site-packages/pandas/core/apply.py", line 1427, in apply
    return self.apply_standard()
  File "/workspaces/KNN-project/.venv/lib/python3.10/site-packages/pandas/core/apply.py", line 1507, in apply_standard
    mapped = obj._map_values(
  File "/workspaces/KNN-project/.venv/lib/python3.10/site-packages/pandas/core/base.py", line 921, in _map_values
    return algorithms.map_array(arr, mapper, na_action=na_action, convert=convert)
  File "/workspaces/KNN-project/.venv/lib/python3.10/site-packages/pandas/core/algorithms.py", line 1743, in map_array


In [ ]:
def extrae_top3(json_list):
    return[actor['name'] for actor in json_list[:3]]

dfUnificada['cast'] = dfUnificada['cast'].apply(extrae_top3)

In [ ]:
def extrae_directores(json_list):
    for member in json_list:
        if member['job'] == 'Director':
            return member['name']
    return None

dfUnificada['director'] = dfUnificada['crew'].apply(extrae_directores)

In [ ]:
dfUnificada['overview'] = dfUnificada['overview'].apply(lambda x: x.split() if isinstance(x, str) else [])

In [ ]:
def remove_spaces(items):
    return [item.replace(" ", "") for item in items]
